In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

batch_size = 100
num_classes = 10
epochs = 150


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#### DATA AUGMENTATION
datagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rotation_range=20,
            zoom_range=[1.0,1.2],
            horizontal_flip=True)

datagen.fit(x_train)

testdatagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
)

testdatagen.fit(x_train)

## DEF A BLOCK CONV VGG + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                  input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  
  return model

  
## DEF VGG TOPOLOGY   
def VGG_model(n):

  if(n==11):
    c1, c2, c3, c4, c5 = 1, 1, 2, 2, 2
  elif(n==13):
    c1, c2, c3, c4, c5 = 2, 2, 2, 2, 2
  elif(n==16):
    c1, c2, c3, c4, c5 = 2, 2, 3, 3, 3
  elif(n==19):
    c1, c2, c3, c4, c5 = 2, 2, 4, 4, 4
  
  model = Sequential()

  for i in range(c1):
    model=CBGN(model,64,x_train.shape[1:])
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  for i in range(c2):
    model=CBGN(model,128)
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  for i in range(c3):
    model=CBGN(model,256)
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  for i in range(c4):
    model=CBGN(model,512)
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  for i in range(c5):
    model=CBGN(model,512)
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(4096))
  model.add(Activation('relu'))

  model.add(Dense(4096))
  model.add(Activation('relu'))

  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  return model
  
model = VGG_model(19)
model.summary()

#### LEARNING RATE SCHEDULER
def scheduler(epoch):
  if epoch < 75:
    return .1
  elif epoch < 125:
    return 0.01
  else:
    return 0.001

set_lr = LRS(scheduler)

### OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


### TRAINING AND VALIDATION
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch=len(x_train) / batch_size, 
          epochs=epochs,
          validation_data=testdatagen.flow(x_test, y_test),
          callbacks=[set_lr],
          shuffle=True,
          verbose=1)

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 32, 32, 64)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
______________